In [1]:
import pandas as pd

# 1) Încarcă toate fișierele cu rezultate adversariale
adv_files = [
    "textFooler/test_textfooler_results_all.csv",
    "PWWS/test_pwws_results_all.csv",
    "textBugger/test_textbugger_results_all.csv",
]


adv_dfs = [pd.read_csv(f) for f in adv_files]
adv_all = pd.concat(adv_dfs, ignore_index=True)

# 2) Filtrează doar atacurile reușite
adv_success = adv_all[adv_all["result_type"] == "Successful"]


In [2]:

# 3) Selectează primele 3000 cu ground_truth_output == 0 și primele 2500 cu == 1
adv0 = adv_success[adv_success["ground_truth_output"] == 0].sample(n=1000, random_state=42)
adv1 = adv_success[adv_success["ground_truth_output"] == 1].sample(n=600, random_state=42)

# 4) Construiește subsetul „curat” exact din original_text
clean0 = adv0[["original_text", "ground_truth_output"]].rename(
    columns={"original_text": "text", "ground_truth_output": "label"}
)
clean1 = adv1[["original_text", "ground_truth_output"]].rename(
    columns={"original_text": "text", "ground_truth_output": "label"}
)

# 5) Construiește subsetul adversarial din perturbed_text
adv0_df = adv0[["perturbed_text", "ground_truth_output"]].rename(
    columns={"perturbed_text": "text", "ground_truth_output": "label"}
)
adv1_df = adv1[["perturbed_text", "ground_truth_output"]].rename(
    columns={"perturbed_text": "text", "ground_truth_output": "label"}
)

# 6) Concatenează curat + adversarial
clean_df = pd.concat([clean0, clean1], ignore_index=True)
adv_df   = pd.concat([adv0_df, adv1_df], ignore_index=True)
train_aug = pd.concat([clean_df, adv_df], ignore_index=True)

# 7) (Opțional) Amestecă datele
train_aug = train_aug.sample(frac=1, random_state=42).reset_index(drop=True)

# 8) Salvează resultatul
train_aug.to_csv("test_augmented.csv", index=False)
print(f"A fost salvat test_augmented.csv cu {len(train_aug)} exemple.")


A fost salvat test_augmented.csv cu 3200 exemple.


In [3]:
import pandas as pd
import re

In [4]:
df = pd.read_csv("test_augmented.csv")

In [5]:
df["text"] = df["text"].str.replace(r"\[\[(.*?)\]\]", r"\1", regex=True)


In [6]:
df["text"].head(10)

0    Despite ostensibly being a boy, local grandson...
1    Avec just hours to go until the third and fіna...
2    Chairman Barack Obama spent various hours on M...
3    George W. Bush Criticizes Obama About War With...
4    Explaining that she had needed a few months to...
5    According to a press conference held timing th...
6    An employee of a USA tech company was arrested...
7    Businessman Donald Trumps failure to insult fe...
8    Malia Obama At The Center Of Damning White Hou...
9    BREAKING: Hillary Faces 10 Years In Prison Ove...
Name: text, dtype: object

In [7]:
df.to_csv("test_augmented.csv", index=False)